In [ ]:
# Now lets download the trained model from the last notebook using joblib
import joblib

# Load the trained model from last notebook
model = joblib.load('../Trained Models/xgboost_model.pkl')

In [ ]:
import shap
import lime
import lime.lime_tabular
import joblib
import numpy as np
import pandas as pd
import xgboost
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
#Just to check the python version and other libraries versions. Remove this cell if everything is working fine..

import platform
print(platform.architecture())

import joblib
print(joblib.__version__)

import xgboost
print(xgboost.__version__)

import platform
import struct
print("Python Version:", platform.python_version())
print("Architecture:", struct.calcsize("P") * 8)

import sys
print(sys.executable)

import struct
print(struct.calcsize("P") * 8)

In [ ]:
data =pd.read_csv('../Data/Processed/compas-scores-two-years-processed.csv')

# Drop the two target columns
X = data.drop('two_year_recid', axis=1)
y = data['two_year_recid']

#model = joblib.load('../Trained Models/xgboost_model.pkl')

model = joblib.load("../Trained Models/xgboost_model.pkl", mmap_mode="r")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
#rf_model.fit(X_train, y_train)

feature_names = X_train.columns.tolist()

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train.values,  # Training data
    training_labels=y_train,  # Labels for training data
    feature_names=feature_names,
    class_names=["No Recidivism", "Recidivism"],  # Adjust based on your target variable
    mode="classification"
)

# Select a sample instance from the test set
i = 5  # Choose any index
instance = X_test.iloc[i].values

# Generate explanation
exp = explainer.explain_instance(instance, model.predict_proba, num_features=10)

# Visualize explanation
exp.show_in_notebook()

In [ ]:
# Create SHAP explainer
explainer = shap.TreeExplainer(model)  # Random Forest is tree-based

# Compute SHAP values
shap_values = explainer.shap_values(X_test)

# Summary plot (global feature importance)
shap.summary_plot(shap_values[1], X_test, feature_names=feature_names)  # Class 1 (Recidivism)

# Explanation for a single instance
shap.force_plot(explainer.expected_value[1], shap_values[1][i], X_test.iloc[i], matplotlib=True)

In [ ]:
# Initialize JavaScript visualization to understand how the model makes decisions (for each feature)
shap.initjs()
shap.force_plot(shap_values[1][0], X_test[0], feature_names=feature_names)